In [1]:
import numpy as np
import gc;from datetime import datetime;import pandas as pd
import warnings;warnings.simplefilter(action='ignore', category=Warning)

In [2]:
train_parquet_path = "train_series.parquet"
test_parquet_path = "test_series.parquet"
train_events_path = "train_events.csv"
train_series = pd.read_parquet(train_parquet_path)
test_series = pd.read_parquet(test_parquet_path)
train_events = pd.read_csv(train_events_path)

In [3]:
series_has_NaN = train_events.groupby('series_id')['step'].apply(lambda x: x.isnull().any())
#series_has_NaN.value_counts()
no_NaN_series = series_has_NaN[~series_has_NaN].index.tolist()
 # incomplete events data
no_NaN_series.remove('31011ade7c0a')
no_NaN_series.remove('a596ad0b82aa') 

In [4]:
def get_train_series(series):
    train_series = pd.read_parquet("train_series.parquet", filters=[('series_id','=',series)])
    train_events = pd.read_csv("train_events.csv").query('series_id == @series')
    train_events = train_events.dropna()
    train_events["step"]  = train_events["step"].astype("int")
    train_events["awake"] = train_events["event"].replace({"onset":1,"wakeup":0})
    train = pd.merge(train_series, train_events[['step','awake']], on='step', how='left')
    train["awake"] = train["awake"].bfill(axis ='rows')
    # final section:
    #train_events.groupby('series_id').tail(1)["event"].unique()
    # Result: the last event is always a "wakeup"
    train['awake'] = train['awake'].fillna(1) # awake
    train["awake"] = train["awake"].astype("int")
    return(train)
clean_train_data = []
for series_id in no_NaN_series:
    train = get_train_series(series_id)
    clean_train_data.append(train)
    del train
    gc.collect();

In [5]:
Zzzs_train = pd.concat(clean_train_data).reset_index(drop=True)
print(Zzzs_train["series_id"].nunique(), "indivudual sleep training series")

35 indivudual sleep training series


In [23]:
file_path = 'C:/temp/clean_zzz.csv'
Zzzs_train.to_csv(file_path, index=False)  # index=False prevents pandas from writing row indices